<a href="https://colab.research.google.com/github/RecoHut-Stanzas/S758139/blob/main/reports/S758139_Report.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# DRGR: Deep Reinforcement learning based Group Recommender system

## Summary Table
| Category | Description |
| --- | --- |
| Problem | Group recommendation problem is challenging because recommending some item that satisfies all the members of the group is rare. So it often involves some compromises that the model has to make in order to maximize the overall satisfaction of the group. |
| Hypothesis | RL agent can learn the required behavior that could the maximize the group's overall satisfaction. |
| Benefits | Meaningful to those people who want to get recommendations for their groups, such as entertainments with families and travels with friends. This model consider the influences of each group member by one self-attention mechanism. |
| Solution | A recommender agent is trained with actor-critic network and is optimized with DDPG algorithm, where the experience replay and target networks are used. Matrix factorization based simulator is built to simulate the MDP environment. It is an extended version of LIRD model for group recommendations. The group recommendation is viewed as a classification task. When one item is recommended to a group, if the group chooses the item, this case is marked as a positive sample. Otherwise, it will be a negative sample. |
| Dataset | MovieLens-1m |
| Preprocessing | Randomly generate groups with 2-5 users. Then, for each group, if every member gives 4-5 stars to one movie, we assume that this movie is adopted by this group with rating 1. If all members give ratings to one movie, but not all in 4-5 stars, we consider the group gives rating 0 to this movie. For other cases, the group movie ratings are missed. Finally, to ensure each group has enough interactions with items, we require each group has at least 20 ratings. Also, for each rating, 100 rating-missed items are randomly sampled. Both user and group rating data are split into training, validation, and testing datasets with the ratio of 70%, 10%, and 20% respectively by the temporal order. |
| Metrics | Recall, nDCG |
| Cluster | PyTorch |
| Tags | ActorCriticNetwork, DDPG, GroupRecommendation, MovieLens1M, PyTorch, ReplayMemory |

## Problem Statement

The recommender system is viewed as an agent, which interacts with the environment $\mathcal{E}$ by recommending items to maximize its reward. The environment is a set of multiple groups, where one single user is assumed to be one member group here. This task is modeled as a Markov Decision Process (MDP). Formally, this MDP contains five elements $\mathcal{(S, A, P, R,\gamma)}$ as follows. Let $\mathcal{U}$, $\mathcal{G}$, and $\mathcal{I}$ be sets of user, group, and item ids respectively.

- State space $\mathcal{S}$: A state $𝑠_𝑡 \in \mathcal{S}$ represents the state of a group at time 𝑡. A state $𝑠_𝑡 = [𝑔, ℎ_𝑡]$ consists of two parts, one for the group id 𝑔 ∈ G and another for the group browsing history $ℎ_𝑡$. The group id 𝑔 can be mapped to its members, i.e. 𝑔 = {𝑢1, 𝑢2, . . . }, where 𝑢1, 𝑢2, · · · ∈ U. The browsing history of a group is ℎ𝑡 = [𝑖1,𝑖2, . . . ,𝑖𝑁 ], where 𝑖1,𝑖2, . . . ,𝑖𝑁 ∈ $\mathcal{I}$.
- Action space $\mathcal{A}$: An action 𝑎𝑡 = [𝑖𝑡,1,𝑖𝑡,2, . . . ,𝑖𝑡,𝐾] ∈ $\mathcal{A}$ is a list of items recommended to a group from the recommender system, where 𝐾 is the number of items. For learning efficiency, we will use 𝐾 = 1 when the agent interacts with the environment, i.e. 𝑎𝑡 ∈ $\mathcal{I}$.
- Reward $\mathcal{R}$: After an action 𝑎𝑡 is taken by the recommender to a group at one state 𝑠𝑡, the recommender will receive a reward 𝑟𝑡 ∈ {0, 1} based on the group response. The reward will be 𝑟𝑡 = 1, if the recommended item is picked by this group. Otherwise, 𝑟𝑡 = 0.
- Transition probability $\mathcal{P}$: Transition probability is defined as $𝑝(𝑠_{𝑡+1} |𝑠_𝑡,𝑎_𝑡)$, which satisfies the Markov property. This probability measures how the environment evolve with the time 𝑡.
- Discount factor 𝛾: The 𝛾 ∈ [0, 1] measures how the future reward will be valuated today.

By given this MDP $\mathcal{(S, A, P, R,\gamma)}$, the agent will try to maximize its rewards by taking actions to interact with the environment. The solution will be the policy 𝜋 : $\mathcal{S}$ → $\mathcal{A}$.

## Modules

### Environment Simulator
<p><center><figure><img src='https://github.com/RecoHut-Stanzas/S758139/raw/main/images/env_sim.png'><figcaption><i>The framework of the environment, where the matrix factorization is used to simulate unknown rewards.</i></figcaption></figure></center></p>

$$h_{t+1} = \begin{cases} [i_2,\dots,i_N,a_t], &\quad r_t \gt0 \\ h_t, &\quad r_t \le 0\end{cases}$$

### Agent Framework
<p><center><figure><img src='https://github.com/RecoHut-Stanzas/S758139/raw/main/images/agent.png'><figcaption><i>The framework of the agent, where state embedding layer, actor network, and critic network are included.</i></figcaption></figure></center></p>

In order to solve the MDP, actor-critic model is built. This agent model consists three networks, i.e. state embedding network, actor network, and critic network.

### State Embedding

The state embedding network is to embed the state $𝑠_𝑡 = [𝑔, ℎ_𝑡]$ to its embedding $s_𝑡$. The group id 𝑔 is mapped to its group members  $𝑔 = \{𝑢_1, 𝑢_2, \dots\}$ first, then those user ids are embedded through the user embedding layer to $\{u_1, u_2, \dots\}$. Then a self-attention user aggregation layer is used to get the embedded group $g$.

Meanwhile, the history $ℎ_𝑡 = [𝑖_1,𝑖_2, \dots,𝑖_𝑁]$ is embedded through the item embedding layer to the embedded history. Combining these two parts, we get the embedded state $s_𝑡 = [g, h_𝑡]$.

### Actor Network

The actor is a multi-layer neural network, whose input is the embedded state $s_𝑡$ and output is the action weight $w_𝑡$. To promote exploration, the Ornstein–Uhlenbeck process noise can be added here. This action weight will take inner products with item embeddings, i.e. $𝑠_𝑗 = w_𝑡^T i_𝑗$. The item with the highest $𝑠_𝑗$ will be recommended to the group, and the corresponding item embedding will be sent to the critic next. The actor network can be optimized by the policy gradient.

### Critic Network

The critic is also a multi-layer neural network, which will assign the Q-value of one state-action pair, i.e. $𝑄(s_𝑡 , a_𝑡)$. The critic network can be optimized by the temporal difference method as the Deep Q-Learning.

## Tutorials
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/RecoHut-Stanzas/S758139/)

### Converting MovieLens-1m data into Group dataset

[direct link to notebook →](https://github.com/RecoHut-Stanzas/S758139/blob/main/nbs/T277604_Group_Data_Generator_on_ML_1m.ipynb)

Randomly generate groups with 2-5 users. Then, for each group, if every member gives 4-5 stars to one movie, we assume that this movie is adopted by this group with rating 1. If all members give ratings to one movie, but not all in 4-5 stars, we consider the group gives rating 0 to this movie. For other cases, the group movie ratings are missed. Finally, to ensure each group has enough interactions with items, we require each group has at least 20 ratings. Also, for each rating, 100 rating-missed items are randomly sampled. Both user and group rating data are split into training, validation, and testing datasets with the ratio of 70%, 10%, and 20% respectively by the temporal order.

**Input files:**

Raw ML-1m dataset downloaded from [https://files.grouplens.org/datasets/movielens/ml-1m.zip](https://files.grouplens.org/datasets/movielens/ml-1m.zip).

**Output files:**

| | |
| --- | --- |
| movies.dat | Movie information file from MovieLens-1M |
| users.dat | User information file from MovieLens-1M |
| groupMember.dat | File including group members. Each line is a group instance: groupID userID1,userID2,... |
| group(user)RatingTrain.dat | Train file. Each line is a training instance: groupID(userID) itemID rating timestamp |
| group(user)RatingVal(Test).dat | group (user) validation (test) file (positive instances). Each line is a validation (test) instance: groupID(userID) itemID rating timestamp |
| group(user)RatingVal(Test)Negative.dat | group (user) validation (test) file (negative instances). Each line corresponds to the line of group(user)RatingVal(Test).dat, containing 100 negative samples. Each line is in the format: (groupID(userID),itemID) negativeItemID1, negativeItemID2, ... |

Interestingly, this can be used as a generic module also to convert user-item dataset into group dataset suitable to train group recommenders. e.g. we can prepare the ML-1m dataset with these 2 commands:

```python
# https://gist.github.com/sparsh-ai/37b36b4024a8e289bbabfcae8ca24bfa/7aad5dca0b028e563a874cce6af2b30f356c8113
!wget -q --show-progress ml1m_groupdata_preprocess.ipynb https://gist.githubusercontent.com/sparsh-ai/37b36b4024a8e289bbabfcae8ca24bfa/raw/7aad5dca0b028e563a874cce6af2b30f356c8113/t277604-group-data-generator-on-ml-1m.ipynb
%run ml1m_groupdata_preprocess.ipynb
```

### DDPG algorithm on the Inverted Pendulum Problem in Keras

[direct link to notebook →](https://github.com/RecoHut-Stanzas/S758139/blob/main/nbs/T085517_Implementing_DDPG_algorithm_on_the_Inverted_Pendulum_Problem_in_Keras.ipynb)

This is an optional tutorial to get us familiarize with the DDPG algorithm. This is taken from the Keras official tutorial collection. In this tutorial, we are solving the inverted pendulum problem using DDPG algorithm.

### Group Recommendation with Actor-Critic Network using DDPG algorithm on ML-1m

[direct link to notebook →](https://github.com/RecoHut-Stanzas/S758139/blob/main/nbs/T381683_Group_Recommendations_with_Actor_critic_RL_Agent_in_MDP_Environment_on_ML_1m_Dataset.ipynb)

## Supplementary Material

### Deep Reinforcement Learning in Recommendation Systems

[https://sparsh-ai.github.io/drl-recsys](https://sparsh-ai.github.io/drl-recsys/R984600_DRL_in_RecSys.html) is a project dedicated to this topic.

### Actor-Critic Network

The actor-network is basically the policy network, and it finds the optimal policy using a policy gradient method. The critic network is the value network, and it estimates the state value.

<img src='https://github.com/RecoHut-Stanzas/S758139/raw/main/images/actorcritic.png'>

The fundamental difference between the REINFORCE and the AC method is that in the REINFORCE, we update the parameter of the network at the end of an episode. But in the AC method, we update the parameter of the network at every step of the episode. 

### DDPG

DDPG, or Deep Deterministic Policy Gradient, is an actor-critic, model-free algorithm based on the deterministic policy gradient that can operate over continuous action spaces. It combines the actor-critic approach with insights from DQNs: in particular, the insights that 1) the network is trained off-policy with samples from a replay buffer to minimize correlations between samples, and 2) the network is trained with a target Q network to give consistent targets during temporal difference backups. DDPG makes use of the same ideas along with batch normalization.

It combines ideas from DPG (Deterministic Policy Gradient) and DQN (Deep Q-Network). It uses Experience Replay and slow-learning target networks from DQN, and it is based on DPG, which can operate over continuous action spaces.

<img src='https://github.com/RecoHut-Stanzas/S758139/raw/main/images/ddpg_algo.png'>

### Experience Replay

We couldn't just blindly apply RL algorithms in a production system out of the box. The learning period would be too costly. Instead, we need to leverage the vast amounts of offline training examples to make the algorithm perform as well as the current system before releasing it into the online production environment.

Both bandit algorithms and A/B tests are online algorithms, which means that they do not have a full set of data upfront to be trained on. Instead, they learn incrementally as the data is accrued. Their performance can be evaluated offline, however, through backtesting using a technique known as the “replay” method. This works by predicting which option the algorithm would select for a viewer, and if there is a historical record of the viewer’s interaction with that option, the result — whether the experience was positive or negative — is counted as if it had happened. If there is no historical record of that interaction, it is ignored.

> **Bandits provide the most benefit when there is a cost associated with making a suboptimal suggestion.**
> 

Bandit’s recommendations will be different from those generated by the model whose recommendations are reflected in your historic dataset. This creates problems that lead to some of the key challenges in evaluating these algorithms using historic data.

1. this is problematic is that your data is probably biased
2. algorithm will often produce recommendations that are different from the recommendations seen by users in the historic dataset

The solution to the above problem is called **replay**. Replay evaluation essentially takes historic events and algorithm’s recommendations at each time step and throws out all samples except for those where your model’s recommendation is the same as the one the user saw in the historic dataset.

**Toy implementation of Replay**

```python
class ReplayMemory(object):
    """
    Replay Memory
    """

    def __init__(self, buffer_size: int):
        """
        Initialize ReplayMemory
        :param buffer_size: size of the buffer
        """
        self.buffer_size = buffer_size
        self.buffer = deque(maxlen=buffer_size)

    def __len__(self):
        return len(self.buffer)

    def push(self, experience: tuple):
        """
        Push one experience into the buffer
        :param experience: (state, action, reward, new_state)
        """
        self.buffer.append(experience)

    def sample(self, batch_size: int):
        """
        Sample one batch from the buffer
        :param batch_size: number of experiences in the batch
        :return: batch
        """
        batch = random.sample(self.buffer, batch_size)
        return batch
```

## Links & References

1. ["DRGR: Deep Reinforcement Learning based Group Recommender System" by Zefang Liu, Shuran Wen, and Yinzhu Quan. arXiv, 2021](https://arxiv.org/abs/2106.06900v1) `paper`
2. [https://github.com/zefang-liu/group-recommender](https://github.com/zefang-liu/group-recommender) `code`
3. [https://github.com/sparsh-ai/stanza/tree/S758139](https://github.com/sparsh-ai/stanza/tree/S758139) `code`